In [1]:
# importing libraries
import numpy as np
import pandas as pd

import os, gc, re

In [2]:
# setting mails dataset directory, change it accordingly
mails_fold = '/Users/atacank/Downloads/maildir'

In [3]:
# reading all subdirectories in mails folder
# importing emails in a list

all_mails_dir_raw = []
all_mails_content = []
saka = []
for root, subdirectories, files in os.walk(mails_fold):
    # for subdirectory in subdirectories:
    #    print(os.path.join(root, subdirectory))
    for file in files:
        # print(os.path.join(root, file))
        try:
            with open(os.path.join(root, file), 'r') as f2:
                data = f2.read()
                all_mails_content.append(data)
            all_mails_dir_raw.append(os.path.join(root, file))
            saka.append([os.path.join(root, file), data])
            
        except:
            pass

In [4]:
import random
# randomly cropping the emails because the data is huge and we only need 3-4k
cropped_mails_dir = random.sample(all_mails_dir_raw, 3974)
cropped_mails_content = random.sample(all_mails_content, 3974)
cropped_data = random.sample(saka, 3974)

In [5]:
enron_df = pd.DataFrame(cropped_data, columns = ['file', 'message'])
df = enron_df

In [7]:
def info_part(i):
    """split infomation part out"""
    return i.split('\n\n', 1)[0]
def content_part(i):
    """split content part out"""
    return i.split('\n\n', 1)[1]

df['pre_info'] = df.message.map(info_part)
df['content'] = df.message.map(content_part)
df['test_true'] = True

words2split = ['Message-ID: ', 'Date: ', 'From: ', 'To: ', 'Subject: ', 'Cc: ', 'Mime-Version: ', 'Content-Type: ',
               'Content-Transfer-Encoding: ', 'Bcc: ', 'X-From: ', 'X-To: ', 'X-cc: ', 'X-bcc: ', 'X-Folder: ', 'X-Origin: ',
               'X-FileName: ']
features_naming = [i[:-2] for i in words2split]
split_condition = '|'.join(words2split)

In [8]:
# Some emails' subject confuse the string-spliting function, so I make a little change
def duplicated_info(i):
    return i.replace(' Date: ', ' Date- ').replace(' Subject: ', ' Subject2: ').replace(' To: ',
                    ' To- ').replace(' (Subject: ', ' (Subject- ')
df['pre_info'] = df['pre_info'].map(duplicated_info)

# checking many categories are there in these emails
def num_part(i):
    return len(re.split(split_condition, i))
df['num_info'] = df['pre_info'].map(num_part)

# around 20k emails do not have the 'To: ' category, so I add one
def add_to(i):
    return i.replace('\nSubject: ', '\nTo: \nSubject: ')
temp_condition = (df['num_info'] == 17) | (df['num_info'] == 15)
df.loc[temp_condition, 'pre_info'] = df.loc[temp_condition, 'pre_info'].map(add_to)


# similar way to deal with the "Cc:" and "Bcc:" categories
temp_condition = (df['num_info'] == 16) | (df['num_info'] == 15)
def add_bcc(i):
    return i.replace('\nX-From: ', '\nBcc: \nX-From: ')
df.loc[temp_condition, 'pre_info'] = df.loc[temp_condition, 'pre_info'].map(add_bcc)
def add_cc(i):
    return i.replace('\nMime-Version: ', '\nCc: \nMime-Version: ')
df.loc[temp_condition, 'pre_info'] = df.loc[temp_condition, 'pre_info'].map(add_cc)

In [9]:
# functions for cleaning tabs and whitespaces
def clearnsh(i):
    return i.replace('\n', ' ')

def cleartsh(i):
    return i.replace('\t', ' ')
df['content'] = df['content'].map(clearnsh)

In [10]:
df['num_info'] = df['pre_info'].map(num_part)
df['num_info'].value_counts()

18    3974
Name: num_info, dtype: int64

In [11]:
df_remove = df.loc[df['num_info'] != 18].copy()
df = df.loc[df['num_info'] == 18].copy()

In [12]:
global feature_idx
def info_split(i):
    ## split the i th part out and remove \n for the feature
    return re.split(split_condition, i)[feature_idx+1][:-2]
def info_split_last(i):
    ## no need to remove \n for last category -- X-FileName
    return re.split(split_condition, i)[feature_idx+1]
for feature_idx in range(len(words2split)):
    if feature_idx != len(words2split) - 1:
        df[features_naming[feature_idx]] = df['pre_info'].map(info_split)
    else:
        df[features_naming[feature_idx]] = df['pre_info'].map(info_split_last) 

In [14]:
df_remove2 = df.loc[df['Content-Transfer-Encoding'] == 'text/plain; charset=us-asci']
df = df.loc[df['Content-Transfer-Encoding'] != 'text/plain; charset=us-asci']

In [16]:
def split_other_content(i):
    """split other forms of contents out"""
    return i.split('-------------', 1)[0]
df["has_other_content"] = df["content"].str.contains("-------------")
df["if_forwarded"] = df["content"].str.contains("------------- Forwarded")
df['content'] = df.content.map(split_other_content)

In [17]:
df = df.drop(['pre_info','test_true', 'num_info'], axis = 1).set_index("file")

In [18]:
ham_emails = pd.DataFrame(columns=['message', 'is_phishing'])
ham_emails['message'] = df['content']
ham_emails['is_phishing'] = 0

In [81]:
ham_emails.to_csv('/Users/atacank/Documents/Repositories/Thesis-Phishing/ham_emails.csv', index=False)